In [1]:
import pandas as pd
import statistics as st
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from sklearn import preprocessing
from catboost import CatBoostClassifier
torch.manual_seed(42)
import xgboost as xgb
import catboost as cb

In [2]:
le=preprocessing.LabelEncoder()
df = pd.read_csv('train.csv')
# df['birth_year']=df['birth_year'].apply(lambda x:year(x))
df['join_year']=pd.DatetimeIndex(df['join_date']).year.values
df['join_month']=pd.DatetimeIndex(df['join_date'],dayfirst=True).month.values
df['join_day']=pd.DatetimeIndex(df['join_date'],dayfirst=True).day.values
# df['sex']=le.fit_transform(df['sex'])
# df['occupation_category_code']=le.fit_transform(df['occupation_category_code'])
# df['occupation_ code']=le.fit_transform(df['occupation_code'])
# df['marital_status']=le.fit_transform(df['marital_status'])
# df['branch_code']=le.fit_transform(df['branch_code'])
df['age']=df['join_year']-df['birth_year']
# df['birth_year']=le.fit_transform(df['birth_year'])
# df['join_year']=le.fit_transform(df['join_year'])
del(df['join_date'])
# del(df['occupation_code'])
df=df[['ID']+list(df.columns.values[-4:])+list(df.columns.values[1:-4])]





In [3]:
df

,ID,join_year,join_month,join_day,age,sex,marital_status,birth_year,branch_code,occupation_code,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,2019.0,2.0,1.0,32.0,F,M,1987,1X1H,2A7I,...,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,2019.0,6.0,1.0,38.0,F,M,1981,UAOD,2A7I,...,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,2013.0,6.0,1.0,22.0,M,U,1991,748L,QZYX,...,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,2019.0,8.0,1.0,29.0,M,M,1990,1X1H,BP09,...,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,2019.0,8.0,1.0,29.0,M,M,1990,748L,NO3L,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29127,8SBA93U,2019.0,9.0,1.0,45.0,F,M,1974,748L,2A7I,...,0,0,0,1,0,0,0,0,0,0
29128,Y73ETTO,2018.0,11.0,1.0,31.0,M,M,1987,XX25,Q6J6,...,0,0,0,1,0,0,0,0,0,0
29129,MVXG2LC,2019.0,2.0,1.0,34.0,M,M,1985,BOAS,6PE7,...,0,0,0,1,0,0,0,0,0,0
29130,BUQ8DQN,2019.0,7.0,1.0,30.0,M,M,1989,UAOD,2A7I,...,0,0,0,1,0,0,0,0,0,0


In [4]:
# le=preprocessing.LabelEncoder()
dt = pd.read_csv('test.csv')
# df['birth_year']=df['birth_year'].apply(lambda x:year(x))
dt['join_year']=pd.DatetimeIndex(dt['join_date']).year.values
dt['join_month']=pd.DatetimeIndex(dt['join_date'],dayfirst=True).month.values
dt['join_day']=pd.DatetimeIndex(dt['join_date'],dayfirst=True).day.values
# dt['sex']=le.fit_transform(dt['sex'])
# dt['occupation_category_code']=le.fit_transform(dt['occupation_category_code'])
# dt['occupation_ code']=le.fit_transform(dt['occupation_code'])
# dt['marital_status']=le.fit_transform(dt['marital_status'])
# dt['branch_code']=le.fit_transform(dt['branch_code'])
dt['age']=dt['join_year']-dt['birth_year']
# df['birth_year']=le.fit_transform(df['birth_year'])
# df['join_year']=le.fit_transform(df['join_year'])
del(dt['join_date'])
# del(df['occupation_code'])
dt=dt[['ID']+list(dt.columns.values[-4:])+list(dt.columns.values[1:-4])]





In [5]:
dt.columns

Index(['ID', 'join_year', 'join_month', 'join_day', 'age', 'sex',
       'marital_status', 'birth_year', 'branch_code', 'occupation_code',
       'occupation_category_code', 'P5DA', 'RIBP', '8NN1', '7POT', '66FJ',
       'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO', 'BSTQ', 'FM3X',
       'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3'],
      dtype='object')

In [6]:
columns = df.columns
policies = columns[11:]
mdf = []
for record in (df['ID']+' '+'X'+' ').values.tolist():
    for policy in policies:
        mdf.append(record+policy)
        
mdf=pd.DataFrame(mdf)
mdf['user']=mdf[0].str.split(' X ',expand=True)[0]
mdf['policy']=mdf[0].str.split(' X ',expand=True)[1]
cdf=mdf.merge(df.iloc[:,:11],left_on='user',right_on='ID')
label=[]
for row,policy in zip(cdf['user'],cdf['policy']):
    label.append(df[df['ID']==row][policy].values[0])
cdf['label']=label
del(cdf['user'])
del(cdf['ID'])
# del(cdt['occupation_code'])

In [7]:
columns = df.columns
policies = columns[11:]
mdt = []
for record in (dt['ID']+' '+'X'+' ').values.tolist():
    for policy in policies:
        mdt.append(record+policy)
        
mdt=pd.DataFrame(mdt)
mdt['user']=mdt[0].str.split(' X ',expand=True)[0]
mdt['policy']=mdt[0].str.split(' X ',expand=True)[1]
cdt=mdt.merge(dt.iloc[:,:11],left_on='user',right_on='ID')
label=[]
for row,policy in zip(cdt['user'],cdt['policy']):
    label.append(dt[dt['ID']==row][policy].values[0])
cdt['label']=label
del(cdt['user'])
del(cdt['ID'])
# del(cdt['occupation_code'])

In [8]:
cdt

,0,policy,join_year,join_month,join_day,age,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,label
0,F86J5PC X P5DA,P5DA,2018.0,12.0,1.0,34.0,M,M,1984,94KC,DZRV,90QI,0
1,F86J5PC X RIBP,RIBP,2018.0,12.0,1.0,34.0,M,M,1984,94KC,DZRV,90QI,0
2,F86J5PC X 8NN1,8NN1,2018.0,12.0,1.0,34.0,M,M,1984,94KC,DZRV,90QI,0
3,F86J5PC X 7POT,7POT,2018.0,12.0,1.0,34.0,M,M,1984,94KC,DZRV,90QI,0
4,F86J5PC X 66FJ,66FJ,2018.0,12.0,1.0,34.0,M,M,1984,94KC,DZRV,90QI,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,WHII93K X JWFN,JWFN,2018.0,11.0,1.0,41.0,M,U,1977,94KC,2A7I,T4MS,0
209996,WHII93K X JZ9D,JZ9D,2018.0,11.0,1.0,41.0,M,U,1977,94KC,2A7I,T4MS,0
209997,WHII93K X J9JW,J9JW,2018.0,11.0,1.0,41.0,M,U,1977,94KC,2A7I,T4MS,0
209998,WHII93K X GHYX,GHYX,2018.0,11.0,1.0,41.0,M,U,1977,94KC,2A7I,T4MS,0


In [49]:
from sklearn.utils import shuffle
cdf = shuffle(cdf)
model = cb.CatBoostClassifier(iterations=5000,task_type='GPU',devices='0:1',learning_rate=0.02)
model.fit(cdf.iloc[:,1:12],cdf['label'],cat_features=['sex','marital_status','branch_code','occupation_code','occupation_category_code','policy'])    
# model.predict_proba(dt.iloc[:,1:9].values)

0:	learn: 0.6576940	total: 140ms	remaining: 11m 37s
1:	learn: 0.6254337	total: 275ms	remaining: 11m 27s
2:	learn: 0.5934443	total: 420ms	remaining: 11m 40s
3:	learn: 0.5647752	total: 552ms	remaining: 11m 29s
4:	learn: 0.5389737	total: 685ms	remaining: 11m 24s
5:	learn: 0.5148471	total: 819ms	remaining: 11m 21s
6:	learn: 0.4905426	total: 953ms	remaining: 11m 19s
7:	learn: 0.4685680	total: 1.09s	remaining: 11m 18s
8:	learn: 0.4480625	total: 1.22s	remaining: 11m 17s
9:	learn: 0.4294424	total: 1.36s	remaining: 11m 19s
10:	learn: 0.4114389	total: 1.5s	remaining: 11m 20s
11:	learn: 0.3940595	total: 1.63s	remaining: 11m 19s
12:	learn: 0.3793875	total: 1.77s	remaining: 11m 18s
13:	learn: 0.3659324	total: 1.91s	remaining: 11m 20s
14:	learn: 0.3527562	total: 2.04s	remaining: 11m 19s
15:	learn: 0.3404610	total: 2.18s	remaining: 11m 18s
16:	learn: 0.3277335	total: 2.31s	remaining: 11m 16s
17:	learn: 0.3174690	total: 2.44s	remaining: 11m 15s
18:	learn: 0.3067529	total: 2.57s	remaining: 11m 14s
19:	

156:	learn: 0.1275361	total: 21.2s	remaining: 10m 54s
157:	learn: 0.1274797	total: 21.4s	remaining: 10m 54s
158:	learn: 0.1274209	total: 21.5s	remaining: 10m 54s
159:	learn: 0.1273552	total: 21.6s	remaining: 10m 54s
160:	learn: 0.1272932	total: 21.8s	remaining: 10m 54s
161:	learn: 0.1272308	total: 21.9s	remaining: 10m 54s
162:	learn: 0.1271714	total: 22s	remaining: 10m 53s
163:	learn: 0.1271193	total: 22.2s	remaining: 10m 53s
164:	learn: 0.1270634	total: 22.3s	remaining: 10m 53s
165:	learn: 0.1270041	total: 22.4s	remaining: 10m 53s
166:	learn: 0.1269483	total: 22.6s	remaining: 10m 53s
167:	learn: 0.1268776	total: 22.7s	remaining: 10m 53s
168:	learn: 0.1268234	total: 22.9s	remaining: 10m 53s
169:	learn: 0.1267355	total: 23s	remaining: 10m 53s
170:	learn: 0.1266858	total: 23.1s	remaining: 10m 53s
171:	learn: 0.1266363	total: 23.3s	remaining: 10m 53s
172:	learn: 0.1265858	total: 23.4s	remaining: 10m 52s
173:	learn: 0.1265441	total: 23.5s	remaining: 10m 52s
174:	learn: 0.1264848	total: 23.

309:	learn: 0.1223425	total: 42.5s	remaining: 10m 42s
310:	learn: 0.1223136	total: 42.6s	remaining: 10m 42s
311:	learn: 0.1223079	total: 42.7s	remaining: 10m 41s
312:	learn: 0.1222833	total: 42.9s	remaining: 10m 41s
313:	learn: 0.1222684	total: 43s	remaining: 10m 41s
314:	learn: 0.1222483	total: 43.1s	remaining: 10m 41s
315:	learn: 0.1222304	total: 43.3s	remaining: 10m 41s
316:	learn: 0.1222205	total: 43.4s	remaining: 10m 41s
317:	learn: 0.1221861	total: 43.6s	remaining: 10m 41s
318:	learn: 0.1221512	total: 43.7s	remaining: 10m 41s
319:	learn: 0.1221229	total: 43.9s	remaining: 10m 41s
320:	learn: 0.1221079	total: 44s	remaining: 10m 41s
321:	learn: 0.1220937	total: 44.1s	remaining: 10m 41s
322:	learn: 0.1220630	total: 44.3s	remaining: 10m 40s
323:	learn: 0.1220551	total: 44.4s	remaining: 10m 40s
324:	learn: 0.1220164	total: 44.5s	remaining: 10m 40s
325:	learn: 0.1219868	total: 44.7s	remaining: 10m 40s
326:	learn: 0.1219514	total: 44.8s	remaining: 10m 40s
327:	learn: 0.1219325	total: 44.

463:	learn: 0.1197865	total: 1m 3s	remaining: 10m 20s
464:	learn: 0.1197659	total: 1m 3s	remaining: 10m 20s
465:	learn: 0.1197423	total: 1m 3s	remaining: 10m 20s
466:	learn: 0.1197194	total: 1m 3s	remaining: 10m 20s
467:	learn: 0.1197159	total: 1m 4s	remaining: 10m 20s
468:	learn: 0.1197062	total: 1m 4s	remaining: 10m 20s
469:	learn: 0.1196936	total: 1m 4s	remaining: 10m 19s
470:	learn: 0.1196869	total: 1m 4s	remaining: 10m 19s
471:	learn: 0.1196638	total: 1m 4s	remaining: 10m 19s
472:	learn: 0.1196580	total: 1m 4s	remaining: 10m 19s
473:	learn: 0.1196545	total: 1m 4s	remaining: 10m 19s
474:	learn: 0.1196343	total: 1m 4s	remaining: 10m 19s
475:	learn: 0.1196293	total: 1m 5s	remaining: 10m 18s
476:	learn: 0.1196163	total: 1m 5s	remaining: 10m 18s
477:	learn: 0.1196102	total: 1m 5s	remaining: 10m 18s
478:	learn: 0.1195890	total: 1m 5s	remaining: 10m 18s
479:	learn: 0.1195777	total: 1m 5s	remaining: 10m 18s
480:	learn: 0.1195596	total: 1m 5s	remaining: 10m 18s
481:	learn: 0.1195542	total:

616:	learn: 0.1181644	total: 1m 24s	remaining: 9m 57s
617:	learn: 0.1181622	total: 1m 24s	remaining: 9m 57s
618:	learn: 0.1181588	total: 1m 24s	remaining: 9m 57s
619:	learn: 0.1181512	total: 1m 24s	remaining: 9m 57s
620:	learn: 0.1181458	total: 1m 24s	remaining: 9m 57s
621:	learn: 0.1181393	total: 1m 24s	remaining: 9m 56s
622:	learn: 0.1181290	total: 1m 24s	remaining: 9m 56s
623:	learn: 0.1181266	total: 1m 25s	remaining: 9m 56s
624:	learn: 0.1181201	total: 1m 25s	remaining: 9m 56s
625:	learn: 0.1181079	total: 1m 25s	remaining: 9m 56s
626:	learn: 0.1180933	total: 1m 25s	remaining: 9m 56s
627:	learn: 0.1180753	total: 1m 25s	remaining: 9m 56s
628:	learn: 0.1180607	total: 1m 25s	remaining: 9m 56s
629:	learn: 0.1180544	total: 1m 25s	remaining: 9m 56s
630:	learn: 0.1180449	total: 1m 26s	remaining: 9m 56s
631:	learn: 0.1180390	total: 1m 26s	remaining: 9m 55s
632:	learn: 0.1180374	total: 1m 26s	remaining: 9m 55s
633:	learn: 0.1180307	total: 1m 26s	remaining: 9m 55s
634:	learn: 0.1180038	total:

...

4251:	learn: 0.1085375	total: 9m 21s	remaining: 1m 38s
4252:	learn: 0.1085371	total: 9m 21s	remaining: 1m 38s
4253:	learn: 0.1085365	total: 9m 22s	remaining: 1m 38s
4254:	learn: 0.1085361	total: 9m 22s	remaining: 1m 38s
4255:	learn: 0.1085355	total: 9m 22s	remaining: 1m 38s
4256:	learn: 0.1085348	total: 9m 22s	remaining: 1m 38s
4257:	learn: 0.1085336	total: 9m 22s	remaining: 1m 38s
4258:	learn: 0.1085333	total: 9m 22s	remaining: 1m 37s
4259:	learn: 0.1085306	total: 9m 22s	remaining: 1m 37s
4260:	learn: 0.1085297	total: 9m 23s	remaining: 1m 37s
4261:	learn: 0.1085293	total: 9m 23s	remaining: 1m 37s
4262:	learn: 0.1085284	total: 9m 23s	remaining: 1m 37s
4263:	learn: 0.1085276	total: 9m 23s	remaining: 1m 37s
4264:	learn: 0.1085272	total: 9m 23s	remaining: 1m 37s
4265:	learn: 0.1085258	total: 9m 23s	remaining: 1m 36s
4266:	learn: 0.1085255	total: 9m 23s	remaining: 1m 36s
4267:	learn: 0.1085249	total: 9m 23s	remaining: 1m 36s
4268:	learn: 0.1085199	total: 9m 24s	remaining: 1m 36s
4269:	lear

4401:	learn: 0.1083483	total: 9m 41s	remaining: 1m 19s
4402:	learn: 0.1083481	total: 9m 41s	remaining: 1m 18s
4403:	learn: 0.1083479	total: 9m 41s	remaining: 1m 18s
4404:	learn: 0.1083477	total: 9m 41s	remaining: 1m 18s
4405:	learn: 0.1083473	total: 9m 42s	remaining: 1m 18s
4406:	learn: 0.1083471	total: 9m 42s	remaining: 1m 18s
4407:	learn: 0.1083463	total: 9m 42s	remaining: 1m 18s
4408:	learn: 0.1083455	total: 9m 42s	remaining: 1m 18s
4409:	learn: 0.1083449	total: 9m 42s	remaining: 1m 17s
4410:	learn: 0.1083431	total: 9m 42s	remaining: 1m 17s
4411:	learn: 0.1083428	total: 9m 42s	remaining: 1m 17s
4412:	learn: 0.1083426	total: 9m 43s	remaining: 1m 17s
4413:	learn: 0.1083424	total: 9m 43s	remaining: 1m 17s
4414:	learn: 0.1083420	total: 9m 43s	remaining: 1m 17s
4415:	learn: 0.1083416	total: 9m 43s	remaining: 1m 17s
4416:	learn: 0.1083413	total: 9m 43s	remaining: 1m 17s
4417:	learn: 0.1083407	total: 9m 43s	remaining: 1m 16s
4418:	learn: 0.1083403	total: 9m 43s	remaining: 1m 16s
4419:	lear

4553:	learn: 0.1082088	total: 10m 1s	remaining: 58.9s
4554:	learn: 0.1082087	total: 10m 1s	remaining: 58.8s
4555:	learn: 0.1082079	total: 10m 1s	remaining: 58.6s
4556:	learn: 0.1082075	total: 10m 1s	remaining: 58.5s
4557:	learn: 0.1082065	total: 10m 1s	remaining: 58.4s
4558:	learn: 0.1082023	total: 10m 2s	remaining: 58.2s
4559:	learn: 0.1082016	total: 10m 2s	remaining: 58.1s
4560:	learn: 0.1082012	total: 10m 2s	remaining: 58s
4561:	learn: 0.1081997	total: 10m 2s	remaining: 57.8s
4562:	learn: 0.1081993	total: 10m 2s	remaining: 57.7s
4563:	learn: 0.1081989	total: 10m 2s	remaining: 57.6s
4564:	learn: 0.1081966	total: 10m 2s	remaining: 57.4s
4565:	learn: 0.1081963	total: 10m 3s	remaining: 57.3s
4566:	learn: 0.1081958	total: 10m 3s	remaining: 57.2s
4567:	learn: 0.1081948	total: 10m 3s	remaining: 57.1s
4568:	learn: 0.1081945	total: 10m 3s	remaining: 56.9s
4569:	learn: 0.1081944	total: 10m 3s	remaining: 56.8s
4570:	learn: 0.1081942	total: 10m 3s	remaining: 56.7s
4571:	learn: 0.1081941	total: 

4704:	learn: 0.1080668	total: 10m 21s	remaining: 39s
4705:	learn: 0.1080667	total: 10m 21s	remaining: 38.8s
4706:	learn: 0.1080664	total: 10m 21s	remaining: 38.7s
4707:	learn: 0.1080660	total: 10m 21s	remaining: 38.6s
4708:	learn: 0.1080614	total: 10m 21s	remaining: 38.4s
4709:	learn: 0.1080610	total: 10m 21s	remaining: 38.3s
4710:	learn: 0.1080609	total: 10m 22s	remaining: 38.2s
4711:	learn: 0.1080605	total: 10m 22s	remaining: 38s
4712:	learn: 0.1080601	total: 10m 22s	remaining: 37.9s
4713:	learn: 0.1080599	total: 10m 22s	remaining: 37.8s
4714:	learn: 0.1080596	total: 10m 22s	remaining: 37.6s
4715:	learn: 0.1080590	total: 10m 22s	remaining: 37.5s
4716:	learn: 0.1080585	total: 10m 22s	remaining: 37.4s
4717:	learn: 0.1080582	total: 10m 23s	remaining: 37.2s
4718:	learn: 0.1080577	total: 10m 23s	remaining: 37.1s
4719:	learn: 0.1080573	total: 10m 23s	remaining: 37s
4720:	learn: 0.1080571	total: 10m 23s	remaining: 36.8s
4721:	learn: 0.1080559	total: 10m 23s	remaining: 36.7s
4722:	learn: 0.1

4854:	learn: 0.1079217	total: 10m 41s	remaining: 19.1s
4855:	learn: 0.1079212	total: 10m 41s	remaining: 19s
4856:	learn: 0.1079210	total: 10m 41s	remaining: 18.9s
4857:	learn: 0.1079202	total: 10m 41s	remaining: 18.8s
4858:	learn: 0.1079198	total: 10m 41s	remaining: 18.6s
4859:	learn: 0.1079197	total: 10m 41s	remaining: 18.5s
4860:	learn: 0.1079192	total: 10m 41s	remaining: 18.4s
4861:	learn: 0.1079179	total: 10m 42s	remaining: 18.2s
4862:	learn: 0.1079177	total: 10m 42s	remaining: 18.1s
4863:	learn: 0.1079174	total: 10m 42s	remaining: 18s
4864:	learn: 0.1079093	total: 10m 42s	remaining: 17.8s
4865:	learn: 0.1079086	total: 10m 42s	remaining: 17.7s
4866:	learn: 0.1079049	total: 10m 42s	remaining: 17.6s
4867:	learn: 0.1079048	total: 10m 42s	remaining: 17.4s
4868:	learn: 0.1078994	total: 10m 43s	remaining: 17.3s
4869:	learn: 0.1078982	total: 10m 43s	remaining: 17.2s
4870:	learn: 0.1078977	total: 10m 43s	remaining: 17s
4871:	learn: 0.1078966	total: 10m 43s	remaining: 16.9s
4872:	learn: 0.1

In [46]:
solution=model.predict_proba(cdt.iloc[:,1:-1].values)[:,1].tolist()

In [47]:
ref=cdt['label']

In [48]:
file=open('answer.txt','w')
for i in range(len(ref)):
    if(ref[i]==0):
        file.write(str(solution[i])+'\n')
    else:
        file.write(str(1)+'\n')
file.close()

In [13]:
df.columns

Index(['ID', 'join_year', 'join_month', 'join_day', 'age', 'sex',
       'marital_status', 'birth_year', 'branch_code', 'occupation_code',
       'occupation_category_code', 'P5DA', 'RIBP', '8NN1', '7POT', '66FJ',
       'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO', 'BSTQ', 'FM3X',
       'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3'],
      dtype='object')

In [14]:
dt

,ID,join_year,join_month,join_day,age,sex,marital_status,birth_year,branch_code,occupation_code,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,F86J5PC,2018.0,12.0,1.0,34.0,M,M,1984,94KC,DZRV,...,0,0,0,0,0,0,0,0,0,0
1,H6141K3,2019.0,10.0,1.0,23.0,M,M,1996,1X1H,J9SY,...,0,0,0,1,0,0,0,0,0,0
2,RBAYUXZ,2020.0,1.0,1.0,52.0,F,W,1968,UAOD,2A7I,...,0,0,0,1,0,0,0,0,0,0
3,KCBILBQ,2019.0,2.0,1.0,30.0,M,M,1989,94KC,2A7I,...,0,0,0,0,0,0,0,0,0,0
4,LSEC1ZJ,2020.0,2.0,1.0,38.0,F,M,1982,UAOD,0KID,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0GMU5UH,2019.0,4.0,1.0,33.0,F,M,1986,748L,2A7I,...,0,0,0,0,1,0,1,0,0,0
9996,I9W11CD,2019.0,8.0,1.0,39.0,M,M,1980,UAOD,0KID,...,0,0,0,0,0,0,0,0,0,0
9997,42WTEGT,2018.0,4.0,1.0,25.0,M,M,1993,748L,SST3,...,0,0,0,1,0,0,0,0,0,0
9998,8EKC4O9,2015.0,5.0,1.0,34.0,F,U,1981,49BM,2A7I,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df

,ID,join_year,join_month,join_day,age,sex,marital_status,birth_year,branch_code,occupation_code,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,2019.0,2.0,1.0,32.0,F,M,1987,1X1H,2A7I,...,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,2019.0,6.0,1.0,38.0,F,M,1981,UAOD,2A7I,...,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,2013.0,6.0,1.0,22.0,M,U,1991,748L,QZYX,...,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,2019.0,8.0,1.0,29.0,M,M,1990,1X1H,BP09,...,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,2019.0,8.0,1.0,29.0,M,M,1990,748L,NO3L,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29127,8SBA93U,2019.0,9.0,1.0,45.0,F,M,1974,748L,2A7I,...,0,0,0,1,0,0,0,0,0,0
29128,Y73ETTO,2018.0,11.0,1.0,31.0,M,M,1987,XX25,Q6J6,...,0,0,0,1,0,0,0,0,0,0
29129,MVXG2LC,2019.0,2.0,1.0,34.0,M,M,1985,BOAS,6PE7,...,0,0,0,1,0,0,0,0,0,0
29130,BUQ8DQN,2019.0,7.0,1.0,30.0,M,M,1989,UAOD,2A7I,...,0,0,0,1,0,0,0,0,0,0


In [16]:
cdt.iloc[:,1:12]

,policy,join_year,join_month,join_day,age,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code
0,P5DA,2018.0,12.0,1.0,34.0,M,M,1984,94KC,DZRV,90QI
1,RIBP,2018.0,12.0,1.0,34.0,M,M,1984,94KC,DZRV,90QI
2,8NN1,2018.0,12.0,1.0,34.0,M,M,1984,94KC,DZRV,90QI
3,7POT,2018.0,12.0,1.0,34.0,M,M,1984,94KC,DZRV,90QI
4,66FJ,2018.0,12.0,1.0,34.0,M,M,1984,94KC,DZRV,90QI
...,...,...,...,...,...,...,...,...,...,...,...
209995,JWFN,2018.0,11.0,1.0,41.0,M,U,1977,94KC,2A7I,T4MS
209996,JZ9D,2018.0,11.0,1.0,41.0,M,U,1977,94KC,2A7I,T4MS
209997,J9JW,2018.0,11.0,1.0,41.0,M,U,1977,94KC,2A7I,T4MS
209998,GHYX,2018.0,11.0,1.0,41.0,M,U,1977,94KC,2A7I,T4MS


In [12]:
cdf

,0,policy,join_year,join_month,join_day,age,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,label
0,4WKQSBB X P5DA,P5DA,2019.0,2.0,1.0,32.0,F,M,1987,1X1H,2A7I,T4MS,0
1,4WKQSBB X RIBP,RIBP,2019.0,2.0,1.0,32.0,F,M,1987,1X1H,2A7I,T4MS,0
2,4WKQSBB X 8NN1,8NN1,2019.0,2.0,1.0,32.0,F,M,1987,1X1H,2A7I,T4MS,0
3,4WKQSBB X 7POT,7POT,2019.0,2.0,1.0,32.0,F,M,1987,1X1H,2A7I,T4MS,0
4,4WKQSBB X 66FJ,66FJ,2019.0,2.0,1.0,32.0,F,M,1987,1X1H,2A7I,T4MS,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
611767,DEJBWYU X JWFN,JWFN,2020.0,3.0,1.0,45.0,M,M,1975,94KC,31JW,90QI,0
611768,DEJBWYU X JZ9D,JZ9D,2020.0,3.0,1.0,45.0,M,M,1975,94KC,31JW,90QI,0
611769,DEJBWYU X J9JW,J9JW,2020.0,3.0,1.0,45.0,M,M,1975,94KC,31JW,90QI,0
611770,DEJBWYU X GHYX,GHYX,2020.0,3.0,1.0,45.0,M,M,1975,94KC,31JW,90QI,0


In [14]:
cdf.iloc[:,1:12].drop(columns='occupation_code')

,policy,join_year,join_month,join_day,age,sex,marital_status,birth_year,branch_code,occupation_category_code
0,P5DA,2019.0,2.0,1.0,32.0,F,M,1987,1X1H,T4MS
1,RIBP,2019.0,2.0,1.0,32.0,F,M,1987,1X1H,T4MS
2,8NN1,2019.0,2.0,1.0,32.0,F,M,1987,1X1H,T4MS
3,7POT,2019.0,2.0,1.0,32.0,F,M,1987,1X1H,T4MS
4,66FJ,2019.0,2.0,1.0,32.0,F,M,1987,1X1H,T4MS
...,...,...,...,...,...,...,...,...,...,...
611767,JWFN,2020.0,3.0,1.0,45.0,M,M,1975,94KC,90QI
611768,JZ9D,2020.0,3.0,1.0,45.0,M,M,1975,94KC,90QI
611769,J9JW,2020.0,3.0,1.0,45.0,M,M,1975,94KC,90QI
611770,GHYX,2020.0,3.0,1.0,45.0,M,M,1975,94KC,90QI
